Advanced LLM Reasoning Techniques - Unit 2.4
=======================================================================

This unit focuses on implementing least-to-most decomposition techniques for solving
complex problems. You'll develop systems that can break down problems into manageable
sub-problems and solve them incrementally. In this exercise, you will also integrate and
compare outputs from Deepseek R1 (Ollama) with ChatOpenAI, enhancing your understanding
through model comparison and didactic analysis.

### Key Concepts to Practice
----------
1. Problem Decomposition
2. Dependency Management
3. Incremental Solution Building
4. Result Integration
5. Solution Verification
6. **Model Comparison:** Evaluate and compare the performance of Deepseek R1 (Ollama) against ChatOpenAI.

Let's build sophisticated incremental problem-solving systems and learn how different LLMs handle decomposition!

## Step 0: Setup and Dependencies
--------------------------------
Ensure all required packages are installed. Uncomment the pip install commands below if necessary.

In [ ]:
!pip install numpy pandas matplotlib langchain openai python-dotenv --quiet
!pip install typing-extensions pydantic pydantic_settings --quiet
!pip install langchain-community langchain-openai --quiet
!pip install -U langchain-ollama --quiet

## Step 1: Initial Configuration
--------------------------------
Import necessary modules and classes for both ChatOpenAI and Deepseek R1 (Ollama).

In [ ]:
from typing import Any, List
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain_ollama import ChatOllama

## Step 1.5: Configuration Management
--------------------------------
Configure API credentials and environment variables for both ChatOpenAI and Deepseek R1.
Make sure to update the API keys in the cell below.

In [ ]:
import os
from pydantic_settings import BaseSettings

Update the API keys below with your actual credentials.

In [ ]:
# Configuration Values - Update these with your API keys
OPENAI_API_KEY = "your-openai-api-key-here"   # Replace with your actual OpenAI API key

Define a Settings class to manage configuration for both LLMs.

In [ ]:
class Settings(BaseSettings):
    """Configuration management for API credentials.

    This class manages API credentials for:
    1. OpenAI (ChatOpenAI)
    2. Ollama (Deepseek R1)

    Attributes:
        openai_api_key: OpenAI API key.
        ollama_api_key: Ollama API key.
        model_name: Identifier for the ChatOpenAI model.
        deepseek_model_name: Identifier for Deepseek R1 from Ollama.
        temperature: Model temperature setting.
    """
    openai_api_key: str = OPENAI_API_KEY
    model_name: str = "gpt-4o-mini-2024-07-18"
    deepseek_model_name: str = "Deepseek R1"  # Adjust as needed for your Ollama configuration
    temperature: float = 0.7

Initialize the environment and create LLM instances for both models.
This function sets environment variables and returns a dictionary containing both LLMs.

In [ ]:
def setup_environment() -> dict:
    """Initialize environment and create LLM instances for model comparison.

    Returns:
        dict: Dictionary with keys 'openai' and 'ollama' containing the respective model instances.
    """
    # Load settings
    settings = Settings()

    # Set environment variables for API keys
    os.environ["OPENAI_API_KEY"] = settings.openai_api_key
    os.environ["OLLAMA_API_KEY"] = settings.ollama_api_key  # if required

    # Initialize ChatOpenAI
    openai_llm = ChatOpenAI(model_name=settings.model_name, temperature=settings.temperature)

    # Initialize ChatOllama for Deepseek R1
    ollama_llm = ChatOllama(model_name=settings.deepseek_model_name, temperature=settings.temperature)

    return {"openai": openai_llm, "ollama": ollama_llm}

Attempt to initialize both LLM instances.
If any errors occur, please verify that your API keys are correctly set.

In [ ]:
# Initialize LLMs
llm_instances = None
try:
    llm_instances = setup_environment()
except Exception as e:
    print(f"Error initializing LLMs: {e}")
    print("Please ensure your API keys are properly set.")

## Usage Instructions:
1. Run the pip install cell (if needed) to install dependencies.
2. Update OPENAI_API_KEY and OLLAMA_API_KEY in the configuration cell.
3. Execute the remaining cells to initialize the environment.
4. Access your LLMs via the `llm_instances` dictionary:
   - `llm_instances['openai']` for ChatOpenAI.
   - `llm_instances['ollama']` for Deepseek R1 (Ollama).

Keep your API keys secure and update settings as necessary.

## Problem 4: Incremental Problem-Solving System
--------------------------------
Design and implement a system that breaks down complex problems into manageable sub-problems
and solves them incrementally. Your system should:

1. **Problem Decomposition:**
   - Break down the main problem into clear sub-problems.
   - Identify logical dependencies among sub-problems.
   - Ensure all aspects of the complex problem are covered.

2. **Solution Strategy:**
   - Solve sub-problems in dependency order.
   - Reuse results where applicable.
   - Track progress and handle error recovery.

3. **Result Integration:**
   - Assemble sub-problem solutions into a final, complete answer.
   - Check for consistency and verify the overall solution.

4. **Model Comparison:**
   - Implement the system to run on both ChatOpenAI and Deepseek R1 (Ollama).
   - Compare differences in decomposition, incremental solving, and final solution quality.
   - Provide commentary on strengths and potential improvements for each model.

### Didactic Enhancements:
- Clearly document each decomposition step and dependency.
- Explain the rationale behind the chosen sub-problems and their ordering.
- Analyze the incremental integration process with detailed commentary.

### Base Classes:
Define a model for sub-problem components.

In [ ]:
class Subproblem(BaseModel):
    """Model for a single sub-problem component.

    Attributes:
        question: The sub-problem question to solve.
        dependencies: List of indices representing dependencies on previous sub-problems.
    """
    question: str = Field(..., description="The sub-problem to solve")
    dependencies: List[int] = Field(..., description="Indices of sub-problems this depends on")

Implement the IncrementalSolver class that decomposes a problem into sub-problems,
solves them in sequence, and integrates the results into a final solution.
This class supports model comparison by allowing the user to choose the LLM to use.

In [ ]:
class IncrementalSolver:
    """A system for solving problems through incremental decomposition.

    This class breaks down complex problems into sub-problems, solves them based on their
    dependencies, and integrates the results into a complete solution. It supports model
    comparison by enabling the use of either ChatOpenAI or Deepseek R1 (Ollama).
    """

    def __init__(self, llm: Any, model_name: str = "openai"):
        """
        Initialize the incremental solving system.

        Args:
            llm: An LLM instance or a dictionary of LLM instances.
            model_name: Choose which model to use ("openai" or "ollama").
        """
        if isinstance(llm, dict):
            self.llm = llm.get(model_name)
            if self.llm is None:
                raise ValueError(f"Model '{model_name}' not found in provided LLM dictionary.")
        else:
            self.llm = llm
        self.model_name = model_name
        # Additional configuration for tracking progress and results can be added here

    def decompose_problem(self, problem: str) -> List[Subproblem]:
        """
        Break down the main problem into ordered sub-problems.

        This method should:
        1. Identify distinct components of the problem.
        2. Establish dependencies between sub-problems.
        3. Return a list of Subproblem objects in dependency order.

        Args:
            problem: The complex problem statement.

        Returns:
            List[Subproblem]: A list of sub-problems with dependencies.
        """
        # TODO: Implement the decomposition logic using self.llm.
        # Example pseudocode:
        # prompt = f"Decompose the following problem into sub-problems with dependencies:\n{problem}"
        # response = self.llm.generate(prompt)
        # Parse response into Subproblem objects.
        pass

    def solve_incrementally(self, problem: str) -> str:
        """
        Solve the problem by addressing sub-problems in dependency order and integrating results.

        This method should:
        1. Decompose the problem into sub-problems.
        2. Solve each sub-problem in order of dependency.
        3. Assemble the intermediate results into a final solution.
        4. Verify the consistency and correctness of the integrated result.

        Args:
            problem: The complex problem statement.

        Returns:
            str: The complete solution after incremental solving.
        """
        # TODO: Implement incremental solving.
        # Pseudocode:
        # subproblems = self.decompose_problem(problem)
        # results = {}
        # for i, sub in enumerate(subproblems):
        #     # Use results from dependencies if needed
        #     prompt = f"Solve the sub-problem: {sub.question}\nUsing previous results: {results}"
        #     result = self.llm.generate(prompt)
        #     results[i] = result
        # final_solution = self.integrate_results(results)
        # return final_solution
        pass

    # Optionally, implement helper methods like integrate_results() and verify_solution().

### Example Test Problems:
Below are sample problems to test your IncrementalSolver system.
These examples illustrate scenarios where complex problems are decomposed,
solved incrementally, and then reassembled into a final solution.

In [ ]:
test_problems = [
    {
        "problem": """
Calculate the total cost of a construction project with the following components:
1. Materials cost $25,000 with 8% tax
2. Labor is $45 per hour for 120 hours
3. Equipment rental is $500 per day for 15 days
4. Insurance is 5% of the subtotal (materials, labor, equipment)
5. Add a 10% contingency to the final total
""",
        "decomposition": [
            {
                "question": "Calculate materials cost including tax",
                "dependencies": [],
                "solution": "Materials with tax = $25,000 × 1.08 = $27,000",
            },
            {
                "question": "Calculate total labor cost",
                "dependencies": [],
                "solution": "Labor cost = $45 × 120 = $5,400",
            },
            {
                "question": "Calculate equipment rental cost",
                "dependencies": [],
                "solution": "Equipment = $500 × 15 = $7,500",
            },
            {
                "question": "Calculate subtotal before insurance",
                "dependencies": [0, 1, 2],
                "solution": "Subtotal = $27,000 + $5,400 + $7,500 = $39,900",
            },
            {
                "question": "Calculate insurance cost",
                "dependencies": [3],
                "solution": "Insurance = $39,900 × 0.05 = $1,995",
            },
            {
                "question": "Calculate total with insurance",
                "dependencies": [3, 4],
                "solution": "Total with insurance = $39,900 + $1,995 = $41,895",
            },
            {
                "question": "Add contingency to final total",
                "dependencies": [5],
                "solution": "Final total = $41,895 × 1.10 = $46,084.50",
            },
        ],
    },
    {
        "problem": """
Write a function to process a list of transactions where each transaction includes a timestamp,
amount, and category. The function should:
1. Filter out invalid transactions (negative amounts)
2. Group transactions by category
3. Calculate total and average for each category
4. Sort categories by total amount
5. Return the top 3 categories with their statistics
""",
        "decomposition": [
            {
                "question": "Define data structures for transaction processing",
                "dependencies": [],
                "solution": "Create a Transaction class and initialize result containers",
            },
            {
                "question": "Implement transaction validation",
                "dependencies": [0],
                "solution": "Filter transactions to keep only those with positive amounts",
            },
            {
                "question": "Group transactions by category",
                "dependencies": [1],
                "solution": "Use a dictionary to group transactions by category",
            },
            {
                "question": "Calculate category statistics",
                "dependencies": [2],
                "solution": "Compute total and average for each category",
            },
            {
                "question": "Sort categories by total amount",
                "dependencies": [3],
                "solution": "Sort the dictionary items by total amount in descending order",
            },
            {
                "question": "Extract top 3 categories with statistics",
                "dependencies": [4],
                "solution": "Select the top 3 categories and format the results",
            },
        ],
    },
]

### Implementation Requirements:

1. **Code Quality:**
   - Provide clear documentation and type hints.
   - Include proper error handling and progress tracking.
   - Ensure each decomposition and integration step is well-documented.

2. **Decomposition Quality:**
   - Logically break down the problem into coherent sub-problems.
   - Clearly define dependencies between sub-problems.
   - Ensure the decomposition covers all aspects of the complex problem.

3. **Testing Approach:**
   - Validate the decomposition and incremental solutions across multiple problem types.
   - Verify that intermediate results are correctly integrated into the final solution.
   - Consider edge cases and potential failure points.

4. **Output Format:**
   - Present sub-problem solutions, progress tracking, and the final integrated solution.
   - Include detailed verification and commentary.
   - Compare outputs from both ChatOpenAI and Deepseek R1 (Ollama) if available.

5. **Model Comparison (Additional Requirement):**
   - Implement the system to support both ChatOpenAI and Deepseek R1.
   - Compare differences in decomposition logic, incremental solving, and result integration.
   - Provide insights and didactic commentary on the strengths and weaknesses of each model.

### Evaluation Criteria:

Your solution will be evaluated based on:
1. Logical decomposition and dependency management.
2. Robustness and correctness of the incremental solution process.
3. Quality of result integration and error handling.
4. Clarity of documentation and didactic commentary.
5. **Model Comparison Insight:** Depth of analysis comparing outputs from ChatOpenAI and Deepseek R1.

### Tips for Success:

1. Clearly define and document each sub-problem.
2. Track and validate dependencies meticulously.
3. Integrate results incrementally with thorough verification.
4. Highlight differences between models and suggest improvements.
5. Document assumptions and rationale at each step.

### Common Pitfalls to Avoid:

1. Circular dependencies or missing steps.
2. Incomplete integration of sub-problem solutions.
3. Poor error handling and lack of progress tracking.
4. Insufficient documentation and didactic analysis.
5. Overlooking differences between the LLMs used.

### Next Steps:

After completing this problem:
1. Explore additional complex problem types.
2. Enhance decomposition and integration strategies.
3. Develop visualization tools for sub-problem dependencies.
4. Implement caching or result reuse mechanisms.
5. Expand your model comparison analysis.